# Futur atlas 2020-2024

In [ ]:
import sys
sys.version

In [ ]:
import os
import re
import pandas as pd
import numpy as np
#from lxml import etree
import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation
#import requests
#import folium
from math import sqrt

# Génération KML pour carrés 1km

In [ ]:
squareCenters = \
[
    (46.108257, 3.071173), # Les Plagnots, Ebreuil (centre équestre)
]

In [ ]:
squareWidth = 1000 # m
squareRadius = sqrt(2*(squareWidth/2)**2)

In [ ]:
kml = skml.Kml()

In [ ]:
lineStyle = skml.LineStyle(color=skml.Color.red, width=1)

In [ ]:
for lat, long in squareCenters:
    ls = kml.newlinestring(coords=kmlcircle.spoints(long=long, lat=lat, meters=squareRadius, n=4, offset=45))
    ls.linestyle = lineStyle

In [ ]:
kml.save('carres-1km.kml')

# Liste des espèces de passereaux à réviser pour les EPOC

En partant de la liste du rapport de synthèse STOC 2001-2019.

In [ ]:
# Nomenclature des espèces
dfEspRef = pd.read_excel('donnees/EspecesRef.ods', engine='odf')
dfEspRef

In [ ]:
import tabula

In [ ]:
tables = tabula.read_pdf('/perso/documents/Ornithologie/STOCTendancesSpecifiques-2001-2019.pdf',
                         pages = "all", multiple_tables = True)
len(tables)

In [ ]:
dfEspEpoc = pd.concat(tables, ignore_index=True)

dfEspEpoc.columns = dfEspEpoc.iloc[:2].apply(lambda sr: ' '.join(str(s) if isinstance(s, str) else '' for s in sr),
                                             axis='index')
dfEspEpoc.columns = ['Espèce'] + list(dfEspEpoc.columns[1:])

dfEspEpoc.drop([0, 1], inplace=True)

dfEspEpoc.loc[dfEspEpoc['Espèce'] == 'Rousserolle effarvate', 'Espèce'] = 'Rousserolle effarvatte'  # M'enfin !?

dfEspEpoc

In [ ]:
dfEspEpoc = dfEspEpoc.join(dfEspRef[['NomFrancaisBis', 'CodeClassif']].drop_duplicates().set_index('NomFrancaisBis'), on='Espèce')
dfEspEpoc.CodeClassif = dfEspEpoc.CodeClassif.astype(int)
dfEspEpoc

In [ ]:
assert dfEspEpoc.CodeClassif.notnull().all()

In [ ]:
dfEspEpoc.to_excel('tmp/EspecesEPOC.ods', index=False)